<a href="https://colab.research.google.com/github/snehachavhan2004/PRODIGY_ML_2/blob/main/PRODIGY_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv('/content/Mall_Customers.csv')

In [ ]:
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [ ]:
df

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


In [ ]:
df.isnull().sum()

,0
CustomerID,0
Gender,0
Age,0
Annual Income (k$),0
Spending Score (1-100),0


In [ ]:
display('duplicated:',df.duplicated().sum())

'duplicated:'

0

In [ ]:
# Step 1: KMeans Clustering
X = df.iloc[:, [3, 4]].values  # Selecting annual income and spending score
wcss = []  # List to store the Within-Cluster Sum of SquaresX=df.iloc[:,[3.4]].values

In [ ]:
# Elbow Method to find the optimal number of clusters
for i in range(1, 11):
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X)
    wcss.append(km.inertia_)

In [ ]:
# Plotting the Elbow Method using Plotly
fig = px.line(x=range(1, 11), y=wcss, title='Elbow Method for Optimal Clusters',
              labels={'x': 'Number of Clusters', 'y': 'WCSS'})
fig.show()

In [ ]:
# Step 2: Apply KMeans with 5 clusters
km = KMeans(n_clusters=5, random_state=0)
y_means = km.fit_predict(X)

In [ ]:
# Step 3: Add cluster labels to the original DataFrame
df['Target'] = y_means

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
 5   Target                  200 non-null    int32 
 6   Cluster                 200 non-null    object
dtypes: int32(1), int64(4), object(2)
memory usage: 10.3+ KB


In [ ]:
# Step 4: Visualize clusters using Plotly
df['Cluster'] = df['Target'].map({0: 'Blue', 1: 'Red', 2: 'Green', 3: 'Yellow', 4: 'Black'})
fig = px.scatter(df, x='Annual Income (k$)', y='Spending Score (1-100)', color='Cluster',
                 title="Clusters of Customers",
                 labels={'annual income (k$)': 'Annual Income (k$)', 'Spending Score (1-100)': 'Spending Score (1-100)'})
fig.show()

In [ ]:
# Step 5: Data Preprocessing
X = df.iloc[:, 1:5]  # Selecting relevant features
y = df.iloc[:, -1]  # Target variable

In [ ]:
# Label encoding for categorical variables (e.g., 'Gender')
le = LabelEncoder()
X['Gender'] = le.fit_transform(X['Gender'])

In [ ]:
# Label encoding and scaling
X['Gender'] = LabelEncoder().fit_transform(X['Gender'])
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]), columns=['Age', 'Annual Income (k$)', 'Spending Score (1-100)'])
X = pd.concat([X_scaled, X[['Gender']]], axis=1)

In [ ]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 6: Training Gradient Boosting Classifier
gbdt = GradientBoostingClassifier(n_estimators=50, random_state=2)
gbdt.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=50, random_state=2)

In [ ]:
# Step 7: Calculate accuracy
accuracy = accuracy_score(y_test, gbdt.predict(X_test))
print(f'Accuracy: {accuracy}')

Accuracy: 0.975


In [ ]:
import plotly.express as px

# Create a DataFrame to compare original vs predicted values
prediction = pd.DataFrame({'Original Value': y_test, 'Predicted Value': y_pred})

# Create a Plotly Express scatter plot to compare original vs predicted values
fig = px.scatter(prediction, x='Original Value', y='Predicted Value',
                 title="Original vs Predicted Values",
                 labels={'Original Value': 'Original Value', 'Predicted Value': 'Predicted Value'},
                 color='Predicted Value', # Optional: Color by predicted value to distinguish them
                 template="plotly_dark")

# Show the plot
fig.show()
